In [ ]:
from copy import deepcopy
from datetime import datetime
from typing import Dict, List

import numpy as np
import tensorflow as tf
from tensorflow import keras

from src.models.geometric_figure import GeometricFigure
from src.services.geometric_figure import (get_geometric_figures,
                                           get_input_and_output)
from src.services.result import save_model_results

In [ ]:
IMAGE_SIZE = (512, 512)
TEST_RATIO = 0.2
DATA_VERSION = '2023-04-03'
MODEL_RESULTS_PATH = 'data/results.json'
NUMBER_OF_REPETITIONS = 10

In [ ]:
ai_models: Dict[str, keras.Model] = {
    'P1': keras.Sequential([
        keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ]),
    'P2': keras.Sequential([
        keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ]),
    'CNN1': keras.Sequential([
        keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((4, 4)),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((4, 4)),
        keras.layers.Flatten(),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ]),
    'CNN2': keras.Sequential([
        keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((4, 4)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((4, 4)),
        keras.layers.Flatten(),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ]),
}

In [ ]:
for name, model in ai_models.items():
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
geometric_figures: List[GeometricFigure] = get_geometric_figures(f'data/{DATA_VERSION}', IMAGE_SIZE, memorize=True)
print(f'Loaded {len(geometric_figures)} geometric figures')

In [ ]:
for i in range(NUMBER_OF_REPETITIONS):
    shuffled_geometric_figures: List[GeometricFigure] = deepcopy(geometric_figures)
    np.random.shuffle(shuffled_geometric_figures)

    x, y = zip(*[get_input_and_output(gf) for gf in shuffled_geometric_figures])
    x = np.array(x)
    y = np.array(y)

    test_size = int(len(x) * TEST_RATIO)
    x_train, x_test = x[:-test_size], x[-test_size:]
    y_train, y_test = y[:-test_size], y[-test_size:]

    for name, model in ai_models.items():
        model.set_weights([np.random.normal(size=w.shape) for w in model.get_weights()])
        print(f'Training {name}')
        model.fit(x_train, y_train, epochs=20)
        print(f'Evaluating {name}')
        loss, accuracy = model.evaluate(x_test, y_test)
        print(f'Saving {name}')
        datetime_now = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        model.save(f'data/models/{DATA_VERSION}/{name}/{datetime_now}.h5')
        save_model_results(name, accuracy, datetime_now, DATA_VERSION, MODEL_RESULTS_PATH)